# GOROKHOV ARTEM

# Fine-tuning VGG Network.

In this hometask you'll need to fine-tune VGG network for dogs classification (the same dataset as in practical seminar).

## Loading the data

In [ ]:
# this cell downloads zip archive with data
! wget "https://www.dropbox.com/s/r11z0ugf2mezxvi/dogs.zip?dl=0" -O dogs.zip

In [ ]:
# this cell extract the archive. You'll now have "dogs" folder in colab
! unzip -qq dogs.zip

## Task 1

Your task is to fine-tune [VGG11 ](https://pytorch.org/vision/0.20/models/generated/torchvision.models.vgg11.html) network from torchvision for the task of dogs breed classification. Your task is to tune the model so that it has the best test accuracy possible. You are not allowed to use any other pretrained model except this and any other data except given.

What you can do:
- **Preprocess and augment data**. Note the following: there is a difference between ordinary data preprocessing (as we did in the practical session) and augmentation. Preprocessing usually refers to the way all the data (train and test) is processed before feeding into the network; augmentation is a technique used to populate training set of samples. Augmentation should only be used on training data, but not on validation and test data. You can read more about augmentation [here](https://d2l.ai/chapter_computer-vision/image-augmentation.html). Also think about what kind of image augmentations are suitable for the given task, e.g. would that be beneficial to flip images vertically in our case?
- **Change/remove/add layers to the network**. You can change layers of the pre-trained VGG11. Note, however, that newly added layers should not be pre-trained. You are allowed to add any layers, e.g. conv, fc, dropout, batchnorm
- **Tune hyperparameters**, e.g. batch size, learning rate, etc.

If X is your score on test set, them your task score is calculated as follows: (min(0.95, X)-0.75)*5

In [3]:
import torch
import numpy as np

def set_seed(seed):
    global SEED
    SEED = seed

    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(37)

In [ ]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.2)
])

valid_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(root="dogs/train", transform=train_transforms)
valid_dataset = ImageFolder(root="dogs/valid", transform=valid_test_transforms)
test_dataset = ImageFolder(root="dogs/test", transform=valid_test_transforms)

batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

num_classes = len(train_dataset.classes)

In [5]:
import torch.nn as nn
from torchvision.models import vgg11

model = vgg11(pretrained=True)

for param in model.features.parameters():
    param.requires_grad = False

for param in model.features[-3:].parameters():
    param.requires_grad = True

model.classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(),
    nn.BatchNorm1d(4096),
    nn.Dropout(0.3),
    
    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.0),
    
    nn.Linear(1024, num_classes)
)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/home/gotheartem/Projects/ioai-hw/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gotheartem/Projects/ioai-hw/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [7]:
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    return 100 * correct / total

In [8]:
from tqdm import tqdm
import torch.optim as optim
from torch.amp import autocast, GradScaler

scaler = GradScaler(device="cuda")

num_epochs = 50
best_val_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")

    for images, labels in train_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        
        with autocast(device_type="cuda"):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        running_loss += loss.item()

        train_bar.set_postfix(loss=running_loss / total, accuracy=100 * correct / total)

    train_acc = 100 * correct / total
    val_acc = evaluate(model, valid_loader)
    scheduler.step()

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_vgg11_model.pth")

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%\n")

Epoch 1/50: 100%|██████████| 32/32 [00:08<00:00,  3.96batch/s, accuracy=28.6, loss=0.0119]


Epoch [1/50], Loss: 2.9571, Train Acc: 28.57%, Val Acc: 70.57%



Epoch 2/50: 100%|██████████| 32/32 [00:07<00:00,  4.09batch/s, accuracy=51.7, loss=0.00787]


Epoch [2/50], Loss: 1.9538, Train Acc: 51.72%, Val Acc: 80.43%



Epoch 3/50: 100%|██████████| 32/32 [00:07<00:00,  4.19batch/s, accuracy=58.1, loss=0.00669]


...

Epoch [48/50], Loss: 1.0504, Train Acc: 73.08%, Val Acc: 92.00%



Epoch 49/50: 100%|██████████| 32/32 [00:07<00:00,  4.08batch/s, accuracy=72.8, loss=0.00415]


Epoch [49/50], Loss: 1.0294, Train Acc: 72.77%, Val Acc: 91.71%



Epoch 50/50: 100%|██████████| 32/32 [00:07<00:00,  4.09batch/s, accuracy=72.8, loss=0.00407]


Epoch [50/50], Loss: 1.0096, Train Acc: 72.75%, Val Acc: 91.71%



In [9]:
model.load_state_dict(torch.load("best_vgg11_model.pth"))

test_acc = evaluate(model, test_loader)
print(f"Test Accuracy: {test_acc:.4f}%")

Test Accuracy: 95.5714%
